In [14]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

In [15]:
import tensorflow as tf
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)

2024-04-30 20:08:37.613727: I external/local_xla/xla/stream_executor/cuda/cuda_gpu_executor.cc:899] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-30 20:08:37.616105: I external/local_xla/xla/stream_executor/cuda/cuda_gpu_executor.cc:899] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-30 20:08:37.617874: I external/local_xla/xla/stream_executor/cuda/cuda_gpu_executor.cc:899] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-b

In [16]:
from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v2')
embeddings = model.encode(sentences)
#print(embeddings)

## load mitre sigma master csv

In [17]:
import pandas as pd 
mitre_sigma = pd.read_csv("MITRE/mitreembed_master_Chroma.csv")


In [18]:
mitre_sigma.Source.value_counts()

Source
SIGMA      3047
MITRE       566
Netflow      13
Name: count, dtype: int64

In [19]:
import pandas as pd
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
# download embeddings model
original_model = SentenceTransformer('all-MiniLM-L12-v2')
# reload model using langchain wrapper
original_model.save('./')

embedding_model_path = './'
embedding_model = HuggingFaceEmbeddings(model_name=embedding_model_path)

In [20]:
#load model
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L12-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L12-v2')



In [21]:
embedding_model = HuggingFaceEmbeddings(model_name=embedding_model_path)

In [22]:
#set chromadb 
#import pydantic
#from pydantic_settings import BaseSettings

from langchain.vectorstores import Chroma
# define logic for embeddings storage
chromadb_path = './'
import chromadb
chroma_client = chromadb.Client()
chroma_client.get_version()

'0.4.10'

In [23]:
from sentence_transformers import SentenceTransformer
 
from langchain.document_loaders import DataFrameLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
import re
#import mlflow
 
import pandas as pd
# assemble product documents in required format (id, text)
documents = (
  DataFrameLoader(
    #CISA_df_pd,
    mitre_sigma,
    page_content_column='Body'
    )
    .load()
  )

In [24]:
# define logic for embeddings storage
chromadb_path = './'

vectordb = Chroma.from_documents(
  documents=documents, 
  embedding=embedding_model, 
  persist_directory=chromadb_path, 
  #collection_name = 'CISA_MITRE'
  )
 
# persist vector db to storage
vectordb.persist()

## check the number of collections in the vectordb

In [25]:
#delete vectors
#vectordb.delete_collection()

#count documents 
vectordb._collection.count()



3626

## Load CISA Advisories

In [26]:
import pandas as pd 
df_CISA = pd.read_csv("CISA_combo_features_new_feb.csv")


In [27]:
#add source field 
df_CISA['Source'] = 'CISA Advisory'

In [31]:
#df_CISA[df_CISA['Body'].str.contains("MITRE")]
df_CISA.dropna()[df_CISA.dropna()['Body'].str.contains('mitre')]


,Subject,Date,Body,filepath,Source
17,Vulnerability Summary for the Week of January ...,"Tue, 30 Jan 2024 11:51:59 +0000",Cybersecurity and Infrastructure Security Agen...,Vulnerability Summary for the Week of January ...,CISA Advisory
28,Vulnerability Summary for the Week of January ...,"Mon, 22 Jan 2024 21:43:22 +0000",Cybersecurity and Infrastructure Security Agen...,Vulnerability Summary for the Week of January ...,CISA Advisory
38,Vulnerability Summary for the Week of January ...,"Thu, 18 Jan 2024 12:15:57 +0000",Cybersecurity and Infrastructure Security Agen...,Vulnerability Summary for the Week of January ...,CISA Advisory
58,Vulnerability Summary for the Week of December...,"Wed, 03 Jan 2024 14:27:32 +0000",Vulnerability Summary for the Week of Dece...,Vulnerability Summary for the Week of December...,CISA Advisory
60,Vulnerability Summary for the Week of December...,"Tue, 26 Dec 2023 23:25:17 +0000",Cybersecurity and Infrastructure Security Agen...,Vulnerability Summary for the Week of December...,CISA Advisory
68,Vulnerability Summary for the Week of December...,"Tue, 19 Dec 2023 13:17:49 +0000",Cybersecurity and Infrastructure Security Agen...,Vulnerability Summary for the Week of December...,CISA Advisory
130,Vulnerability Summary for the Week of October ...,"Mon, 06 Nov 2023 22:48:34 +0000",Cybersecurity and Infrastructure Security Agen...,Vulnerability Summary for the Week of October ...,CISA Advisory
147,Vulnerability Summary for the Week of October ...,"Tue, 24 Oct 2023 16:05:45 +0000",Cybersecurity and Infrastructure Security Agen...,Vulnerability Summary for the Week of October ...,CISA Advisory
181,Vulnerability Summary for the Week of Septembe...,"Tue, 03 Oct 2023 12:26:33 +0000",Cybersecurity and Infrastructure Security Agen...,Vulnerability Summary for the Week of Septembe...,CISA Advisory
211,Vulnerability Summary for the Week of Septembe...,"Mon, 11 Sep 2023 20:50:46 +0000",Cybersecurity and Infrastructure Security Agen...,Vulnerability Summary for the Week of Septembe...,CISA Advisory


In [20]:
from sentence_transformers import SentenceTransformer
 
from langchain.document_loaders import DataFrameLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
import re
#import mlflow
 
import pandas as pd
# assemble product documents in required format (id, text)
documents = (
  DataFrameLoader(
    #CISA_df_pd,
    df_CISA,
    page_content_column='Body'
    )
    .load()
  )

In [21]:
# define logic for embeddings storage
chromadb_path = './'

vectordb = Chroma.from_documents(
  documents=documents, 
  embedding=embedding_model, 
  persist_directory=chromadb_path, 
  #collection_name = 'CISA_MITRE'
  )
 
# persist vector db to storage
vectordb.persist()

In [40]:
#delete vectors
#vectordb.delete_collection()

#count documents 
vectordb._collection.count()



7771

In [22]:
df_CISA.head(1)

,Subject,Date,Body,filepath,Source
0,CISA Partners With OpenSSF Securing Software R...,"Thu, 08 Feb 2024 19:16:35 +0000",CISA Partners With OpenSSF Securing Softwa...,CISA Partners With OpenSSF Securing Software R...,CISA Advisory


In [32]:
#examine a vector db record
rec= vectordb._collection.peek(1)
print('Metadatas:  ', rec['metadatas'])
print('Documents:  ', rec['documents'])
print('ids:        ', rec['ids']) 
print('embeddings: ', rec['embeddings'])

Metadatas:   [{'Date': '4/5/24', 'Source': 'MITRE', 'Subject': 'Analyze network data for uncommon data flows (e.g., a client sending significantly more data than it receives from a server). Processes utilizing the network that do not normally have network communication or have never been seen before are suspicious. Analyze packet contents to detect communications that do not follow the expected protocol behavior for the port that is being used. (Citation: University of Birmingham C2)', 'filepath': 'https://attack.mitre.org/techniques/T1001'}]
Documents:   ['Adversaries may obfuscate command and control traffic to make it more difficult to detect. Command and control (C2) communications are hidden (but not necessarily encrypted) in an attempt to make the content more difficult to discover or decipher and to make the communication less conspicuous and hide commands from being seen. This encompasses many methods, such as adding junk data to protocol traffic, using steganography, or impers

In [33]:
#results = vectordb._collection.query(include=['embeddings'])
#results_json = vectordb.get()
results_json = vectordb.get(include=['embeddings', 'documents', 'metadatas'])



In [34]:
import json
#data = json.loads(results_json).astype(str)
#embeddings = json.dumps(data)
#pd.read_json(embeddings)
embeddings = pd.DataFrame.from_dict(results_json)


In [35]:
embeddings.to_csv('mitre_embeddings.csv')

In [36]:
vectordb.similarity_search_with_score("MoveIt Vulnerability")

[(Document(page_content="Adversaries may move onto systems, possibly those on disconnected or air-gapped networks, by copying malware to removable media and taking advantage of Autorun features when the media is inserted into a system and executes. In the case of Lateral Movement, this may occur through modification of executable files stored on removable media or by copying malware and renaming it to look like a legitimate file to trick users into executing it on a separate system. In the case of Initial Access, this may occur through manual manipulation of the media, modification of systems used to initially format the media, or modification to the media's firmware itself.", metadata={'Date': '4/5/24', 'Source': 'MITRE', 'Subject': 'Monitor file access on removable media. Detect processes that execute from removable media after it is mounted or when initiated by a user. If a remote access tool is used in this manner to move laterally, then additional actions are likely to occur after

In [28]:
##vectordb.similarity_search_with_relevance_scores("MoveIt Vulnerability", k=50, score_threshold=0.80, search_type="hybrid")


In [37]:
class ProductSearchWrapper:
    def __init__(self, embedding_model, chromadb, max_results=25):
        # Retrieve embedding model
        self.embedding_model = HuggingFaceEmbeddings(model_name=embedding_model_path)

        # Retrieve vectordb contents
        self._vectordb = Chroma(persist_directory=chromadb_path, embedding_function=self.embedding_model)

        # Set number of results to returnd
        self._max_results = max_results

    def predict(self, query):
        # Perform search on embeddings
        raw_results = self._vectordb.similarity_search_with_score(query, k=self._max_results)

 
        # get lists of of scores, descriptions and ids from raw results
        scores, Body, Subject, Date, filepath, Source = zip(
          *[(r[1], r[0].page_content, r[0].metadata['Subject'], r[0].metadata['Date'], r[0].metadata['filepath'], 
             r[0].metadata['Source']) 
             for r in raw_results] 
          )
        results_pd = pd.DataFrame({        
            #'Body':Body,
            'Subject':Subject,
            'Date':Date,
            'Body':Body,
            'filepath':filepath,
            'Source':Source,
            'score':scores
          }).sort_values(axis=0, by='score', ascending=True)

        #set return value
        return results_pd
        #return raw_results


In [38]:
# Instantiate the model
#model objective:
"""
The model is intented to be used as a sentence and short paragraph encoder. 
Given an input text, it ouptuts a vector which captures the semantic information. 
The sentence vector may be used for information retrieval, clustering or sentence similarity tasks.
"""

model = ProductSearchWrapper(embedding_model=embedding_model_path, chromadb=chromadb_path, max_results=25)


# Call the model's predict method with a query
#query = "Which is the most recent CISA advisory on Ivanti"
query = "MoveIT vulnerability"
results = model.predict(query)

#top 5 results
df_results = pd.DataFrame(results).head(5)
#df_results

#df_results.to_csv('result_test.csv')
# response_text = df_results[['Subject', 'Date','filepath','score']]
# response_text
#str(response_text.iloc[0]['Subject'])

#df_results.iloc[0]
#df_results.iloc[0]['Body']
df_results

,Subject,Date,Body,filepath,Source,score
0,Monitor file access on removable media. Detect...,4/5/24,"Adversaries may move onto systems, possibly th...",https://attack.mitre.org/techniques/T1091,MITRE,1.171260
1,Monitor for file creation and files transferre...,4/5/24,Adversaries may transfer tools or other files ...,https://attack.mitre.org/techniques/T1105,MITRE,1.255052
2,Monitor for file creation and files transferre...,4/5/24,Adversaries may transfer tools or other files ...,https://attack.mitre.org/techniques/T1570,MITRE,1.278562
3,Processes that write or overwrite many files t...,4/5/24,\nAdversaries may deliver payloads to remote s...,https://attack.mitre.org/techniques/T1080,MITRE,1.322089
4,Monitor for common command-line editors used t...,4/5/24,Adversaries can modify property list files (pl...,https://attack.mitre.org/techniques/T1547/011,MITRE,1.401985


## LLM integration

In [39]:
import torch
use_cuda = torch.cuda.is_available()
if use_cuda:
    print('__CUDNN VERSION:', torch.backends.cudnn.version())
    print('__Number CUDA Devices:', torch.cuda.device_count())
    print('__CUDA Device Name:',torch.cuda.get_device_name(0))
    print('__CUDA Device Total Memory [GB]:',torch.cuda.get_device_properties(0).total_memory/1e9)

__CUDNN VERSION: 8904
__Number CUDA Devices: 1
__CUDA Device Name: Tesla T4
__CUDA Device Total Memory [GB]: 15.655829504


In [42]:
import gc
import torch

gc.collect()

torch.cuda.empty_cache()

In [43]:
## initialize LLM wizardlm-13b
from langchain.llms import LlamaCpp
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'


In [44]:
n_gpu_layers = 35  # Metal set to 1 is enough.
#n_batch = 1042  # Should be between 1 and n_ctx, consider the amount of RAM of your Apple Silicon Chip.
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

# Make sure the model path is correct for your system!
llm = LlamaCpp(
    #model_path="Lilly/Lily-7B-Instruct-v0.2.Q5_K_M.gguf",
    model_path="./wizardlm-13b-v1.1-superhot-8k.ggmlv3.q4_0.gguf.bin",
    n_gpu_layers=n_gpu_layers,
 #   n_batch=n_batch,
 #   temperature=0.75,
    max_tokens=2000,
 #   top_p=1,
    n_ctx=2048,
    f16_kv=True,  # MUST set to True, otherwise you will run into problem after a couple of calls
    callback_manager=callback_manager,
    verbose=False,
)

ggml_init_cublas: GGML_CUDA_FORCE_MMQ:   no
ggml_init_cublas: CUDA_USE_TENSOR_CORES: yes
ggml_init_cublas: found 1 CUDA devices:
  Device 0: Tesla T4, compute capability 7.5, VMM: yes


In [45]:
#%tensorflow_version 2.x
#!pip install tf-nightly
import tensorflow as tf
device_name = tf.test.gpu_device_name()
print(device_name)

/device:GPU:0


2024-04-30 20:14:39.645178: I external/local_xla/xla/stream_executor/cuda/cuda_gpu_executor.cc:899] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-30 20:14:39.647554: I external/local_xla/xla/stream_executor/cuda/cuda_gpu_executor.cc:899] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-30 20:14:39.649300: I external/local_xla/xla/stream_executor/cuda/cuda_gpu_executor.cc:899] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-b

## intialize koboldcpp

In [47]:
#https://medium.com/p/9f890e6960f3
# Successfully uninstalled langchain-0.0.198

from langchain import PromptTemplate, LLMChain


vectordb = Chroma(persist_directory=chromadb_path, embedding_function=embedding_model)
#retrievers

retriever = vectordb.as_retriever()

from langchain.chains import RetrievalQA

# qa_chain = RetrievalQA.from_chain_type(
#     llm,
#     retriever=vectordb.as_retriever()
# )

# Build prompt
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer. 
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)# Run chain

from langchain.chains import RetrievalQA, ConversationalRetrievalChain
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="refine",
    #chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
    retriever=vectordb.as_retriever(search_kwargs={"k": 1}),
    return_source_documents=True,
    verbose=False,
)
#from langchain.llms import KoboldApiLLM
#llm = KoboldApiLLM(endpoint="http://localhost:5001", max_length=80)
#llm = KoboldApiLLM(endpoint="http://54.165.180.47:5001", max_length=80)

In [48]:
res = qa("Explain to me what  Kerberoasting is and find a MITRE technique associated with it")
res

---------------------------------------------------------------------------------------------------------

{'query': 'Explain to me what  Kerberoasting is and find a MITRE technique associated with it',
 'result': '---------------------------------------------------------------------------------------------------------',
 'source_documents': [Document(page_content='Adversaries may attempt to subvert Kerberos authentication by stealing or forging Kerberos tickets to enable [Pass the Ticket](T1550.003). Kerberos is an authentication protocol widely used in modern Windows domain environments. In Kerberos environments, referred to as ‚Äúrealms‚Äù, there are three basic participants: client, service, and Key Distribution Center (KDC).(Citation: ADSecurity Kerberos Ring Decoder) Clients request access to a service and through the exchange of Kerberos tickets, originating from KDC, they are granted access after having successfully authenticated. The KDC is responsible for both authentication and ticket granting.  Attackers may attempt to abuse Kerberos by stealing tickets or forging tickets to ena

In [50]:
res = qa("  Below is a script to look through logs for malicious events, explain to me the MITRE technique for which this detection b belongs to \
remoteconnectionmanager` EventCode=1149 \
| stats count min(_time) as firstTime max(_time) as lastTime by ComputerName, Source_Network_Address, User, Message \
| `security_content_ctime(firstTime)` \
| `security_content_ctime(lastTime)` \
| rename ComputerName as dest \
| `windows_rdp_connection_successful_filter")
res


1. The MITRE technique being used here is `remoteconnectionmanager`.

{'query': '  Below is a script to look through logs for malicious events, explain to me the MITRE technique for which this detection b belongs to remoteconnectionmanager` EventCode=1149 | stats count min(_time) as firstTime max(_time) as lastTime by ComputerName, Source_Network_Address, User, Message | `security_content_ctime(firstTime)` | `security_content_ctime(lastTime)` | rename ComputerName as dest | `windows_rdp_connection_successful_filter',
 'result': '\n1. The MITRE technique being used here is `remoteconnectionmanager`.',
 'source_documents': [Document(page_content='Adversaries may check for Internet connectivity on compromised systems. This may be performed during automated discovery and can be accomplished in numerous ways such as using [Ping](S0097), `tracert`, and GET requests to websites.\n\nAdversaries may use the results and responses from these requests to determine if the system is capable of communicating with their C2 servers before attempting to connect to them. T

In [69]:
res = qa("Explain to me Which MITRE technique does this detection belongs to \
powershell EventCode=4104 ScriptBlockText = *Net.webclient* AND ScriptBlockText = *.UploadString* \
| stats count min(_time) as firstTime max(_time) as lastTime by EventCode ScriptBlockText Computer UserID \
| security_content_ctime(firstTime) \
| security_content_ctime(lastTime) \
| windows_exfiltration_over_c2_via_powershell_uploadstring_filter")
res




The detection belongs to MITRE TECHNICAL OBJECT CONCEPT (TOCTOU) technique. The command is using PowerShell's `System.Net.WebClient` class along with `UploadString` method to download and execute potentially malicious code from remote locations. The script then calculates the first and last occurrence times of such events using the `security_content_ctime()` function, which could indicate continuous exfiltration of sensitive data or content over time via C2 communication channels established via PowerShell and `UploadString`.

{'query': 'Explain to me Which MITRE technique does this detection belongs to powershell EventCode=4104 ScriptBlockText = *Net.webclient* AND ScriptBlockText = *.UploadString* | stats count min(_time) as firstTime max(_time) as lastTime by EventCode ScriptBlockText Computer UserID | security_content_ctime(firstTime) | security_content_ctime(lastTime) | windows_exfiltration_over_c2_via_powershell_uploadstring_filter',
 'result': "\nThe detection belongs to MITRE TECHNICAL OBJECT CONCEPT (TOCTOU) technique. The command is using PowerShell's `System.Net.WebClient` class along with `UploadString` method to download and execute potentially malicious code from remote locations. The script then calculates the first and last occurrence times of such events using the `security_content_ctime()` function, which could indicate continuous exfiltration of sensitive data or content over time via C2 communication channels established via PowerShell and `UploadString`.",
 'source_documents': [Document(

In [74]:
res = qa("Explain to me Which MITRE technique does this detection belongs to \
| tstats security_content_summariesonly count min(_time) as firstTime max(_time) as lastTime values(All_Traffic.dest_port) as dest_port latest(user) as user from datamodel=Network_Traffic by All_Traffic.src All_Traffic.dest, All_Traffic.app \
| drop_dm_object_name(All_Traffic) \
| security_content_ctime(firstTime) \
| security_content_ctime(lastTime) \
| lookup remote_access_software remote_appid AS app OUTPUT isutility, description as signature, comment_reference as desc, category \
| search isutility = True \
| detect_remote_access_software_usage_traffic_filter ")
res



| filter bysource(All_Traffic.src) | filter bysrcport(All_Traffic.dest_port) | filter by(app=app) | transform value(signature) AS app_name | rename app_name as app_name | rename desc as category | rename desc as signature | rename desc as desc | rename desc as desc | rename desc as desc | rename desc as desc | rename desc as desc | rename desc as desc

{'query': 'Explain to me Which MITRE technique does this detection belongs to | tstats security_content_summariesonly count min(_time) as firstTime max(_time) as lastTime values(All_Traffic.dest_port) as dest_port latest(user) as user from datamodel=Network_Traffic by All_Traffic.src All_Traffic.dest, All_Traffic.app | drop_dm_object_name(All_Traffic) | security_content_ctime(firstTime) | security_content_ctime(lastTime) | lookup remote_access_software remote_appid AS app OUTPUT isutility, description as signature, comment_reference as desc, category | search isutility = True | detect_remote_access_software_usage_traffic_filter ',
 'result': '| filter bysource(All_Traffic.src) | filter bysrcport(All_Traffic.dest_port) | filter by(app=app) | transform value(signature) AS app_name | rename app_name as app_name | rename desc as category | rename desc as signature | rename desc as desc | rename desc as desc | rename desc as desc | rename desc as desc | rename desc as desc | rename desc as 

In [75]:
res = qa("Explain to me Which MITRE technique does this detection belongs to \
| tstats security_content_summariesonly count from datamodel=Network_Traffic where All_Traffic.dest_port=139 OR All_Traffic.dest_port=445 OR All_Traffic.app=smb by _time span=1h, All_Traffic.src \
| drop_dm_object_name(All_Traffic) \
| eventstats max(_time) as maxtime \
| stats count as num_data_samples max(eval(if(_time >= relative_time(maxtime, -70m@m), count, null))) as count avg(eval(if(_time<relative_time(maxtime, -70m@m), count, null))) as avg stdev(eval(if(_time<relative_time(maxtime, -70m@m), count, null))) as stdev by src \
| eval upperBound=(avg+stdev*2), isOutlier=if(count > upperBound AND num_data_samples >=50, 1, 0) \
| where isOutlier=1 \
| table src count \
| smb_traffic_spike_filter  ")
res

| filter  
| smb_traffic_spike_filter  
| smb  
| port 139  
| port 445  
| time span 1h  

{'query': 'Explain to me Which MITRE technique does this detection belongs to | tstats security_content_summariesonly count from datamodel=Network_Traffic where All_Traffic.dest_port=139 OR All_Traffic.dest_port=445 OR All_Traffic.app=smb by _time span=1h, All_Traffic.src | drop_dm_object_name(All_Traffic) | eventstats max(_time) as maxtime | stats count as num_data_samples max(eval(if(_time >= relative_time(maxtime, -70m@m), count, null))) as count avg(eval(if(_time<relative_time(maxtime, -70m@m), count, null))) as avg stdev(eval(if(_time<relative_time(maxtime, -70m@m), count, null))) as stdev by src | eval upperBound=(avg+stdev*2), isOutlier=if(count > upperBound AND num_data_samples >=50, 1, 0) | where isOutlier=1 | table src count | smb_traffic_spike_filter  ',
 'result': '| filter  \n| smb_traffic_spike_filter  \n| smb  \n| port 139  \n| port 445  \n| time span 1h  ',
 'source_documents': [Document(page_content='PortScan', metadata={'Date': '7/7/17 1:05', 'Source': 'Netflow', 'Sub

In [76]:
res = qa("Explain to me Which MITRE technique does this detection belongs to \
powershell EventCode=4104 ScriptBlockText = *Net.webclient* AND ScriptBlockText = *.UploadString* \
| stats count min(_time) as firstTime max(_time) as lastTime by EventCode ScriptBlockText Computer UserID \
| security_content_ctime(firstTime) \
| security_content_ctime(lastTime) \
| windows_exfiltration_over_c2_via_powershell_uploadstring_filter")
res




The detection belongs to MITRE technique TP-SA-005 - Powershell Malware. The provided PowerShell code uses the `System.Net.WebClient` class to make an HTTP request, specifically using the `UploadString` method, which suggests that it might be used for exfiltration activities. The output of this detection shows various statistical information related to the usage of this technique on different computers and users, including the first and last time the technique was used on each computer, as well as the exfiltration activity duration. The `security_content_ctime(firstTime)` and `security_content_ctime(lastTime)` functions are used to display the creation time of the relevant security content files, while `windows_exfiltration_over_c2_via_powershell_uploadstring_filter` suggests that the detected activity might be related to exfiltration over a C2 channel using PowerShell's `UploadString` method.

{'query': 'Explain to me Which MITRE technique does this detection belongs to powershell EventCode=4104 ScriptBlockText = *Net.webclient* AND ScriptBlockText = *.UploadString* | stats count min(_time) as firstTime max(_time) as lastTime by EventCode ScriptBlockText Computer UserID | security_content_ctime(firstTime) | security_content_ctime(lastTime) | windows_exfiltration_over_c2_via_powershell_uploadstring_filter',
 'result': "\nThe detection belongs to MITRE technique TP-SA-005 - Powershell Malware. The provided PowerShell code uses the `System.Net.WebClient` class to make an HTTP request, specifically using the `UploadString` method, which suggests that it might be used for exfiltration activities. The output of this detection shows various statistical information related to the usage of this technique on different computers and users, including the first and last time the technique was used on each computer, as well as the exfiltration activity duration. The `security_content_ctim

In [81]:
res = qa("Explain to me Which MITRE technique does this detection belongs to \
Domain Controller\
windows_ad_rogue_domain_controller_network_activity_filter\
")
res



------------------
This detection belongs to MITRE TECHNIQUE \_\_\_\_\_\_\_\_\_\_Domain Controller \_\_\_\_\_\_\_\_\_\_Windows_AD_Rogue_Domain_Controller_Network_Activity_Filter

{'query': 'Explain to me Which MITRE technique does this detection belongs to Domain Controllerwindows_ad_rogue_domain_controller_network_activity_filter',
 'result': '------------------\nThis detection belongs to MITRE TECHNIQUE \\_\\_\\_\\_\\_\\_\\_\\_\\_\\_Domain Controller \\_\\_\\_\\_\\_\\_\\_\\_\\_\\_Windows_AD_Rogue_Domain_Controller_Network_Activity_Filter',
 'source_documents': [Document(page_content='Adversaries may check for Internet connectivity on compromised systems. This may be performed during automated discovery and can be accomplished in numerous ways such as using [Ping](S0097), `tracert`, and GET requests to websites.\n\nAdversaries may use the results and responses from these requests to determine if the system is capable of communicating with their C2 servers before attempting to connect to them. The results may also be used to identify routes, redirectors, and proxy servers.', metadata={'Date': '4/5/24', 'Source': 'MITRE', 'Subject': 'System and network discovery

In [52]:
#!pip install langchain langchain-openai chromadb renumics-spotlight
from renumics import spotlight
#spotlight.show(mitre_sigma)
spotlight.show(mitre_sigma, dtype={"image": spotlight.Image, "embedding": spotlight.Embedding})


